In [1]:
from transformers import pipeline
from datasets import load_dataset
import jsonlines
from tqdm.auto import tqdm

In [3]:
data_name = "SetFit/bbc-news"
dataset = load_dataset(data_name, split="train")

Using custom data configuration SetFit--bbc-news-dbeb222bfdd4d6ed
Found cached dataset json (C:/Users/heegyukim/.cache/huggingface/datasets/SetFit___json/SetFit--bbc-news-dbeb222bfdd4d6ed/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


Measure Data Mean Token Length

In [4]:
from transformers import AutoTokenizer

model_name = 'abhishek/autonlp-bbc-news-classification-37229289'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
import pandas as pd
lengths = [len(tokenizer.encode(x, truncation=False)) for x in dataset['text']]

Token indices sequence length is longer than the specified maximum sequence length for this model (706 > 512). Running this sequence through the model will result in indexing errors


In [6]:
lengths= pd.Series(lengths)

In [7]:
lengths.describe()

count    1225.000000
mean      460.301224
std       304.657820
min       104.000000
25%       293.000000
50%       397.000000
75%       556.000000
max      5032.000000
dtype: float64

In [8]:
device = 'cuda:0'
model = pipeline('text-generation', 'gpt2-medium', device=device)

Downloading: 100%|██████████| 718/718 [00:00<00:00, 359kB/s]
d:\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\heegyukim\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 1.52G/1.52G [02:13<00:00, 11.4MB/s]
Downloading: 100%|██████████| 

In [11]:
classifier = pipeline('text-classification', model='abhishek/autonlp-bbc-news-classification-37229289', device=device)

In [9]:
filename = data_name.replace("/", "__")
with jsonlines.open(f"data/gpt2-medium-{filename}.jsonl", 'w') as f:
    for item in tqdm(dataset):
        text = item['text'].strip()
        if len(text) >= 1024:
            text = text[-1024:]
        gens = model(text, max_new_tokens=400, num_return_sequences=5)
        gens = [g['generated_text'][len(text):] for g in gens]
        item['generation'] = gens
        # item['prediction'] = classifier(gens)
        f.write(item)
        # print(text)
        # print(item['label'])
        # print(gens)
        # print(classifier(gens))

  1%|          | 10/1225 [01:38<3:02:17,  9.00s/it]d:\Anaconda3\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 1225/1225 [2:54:48<00:00,  8.56s/it]


In [ ]:
import jsonlines
from collections import defaultdict

label2id = {
  "business": 0,
  "entertainment": 1,
  "politics": 2,
  "sport": 3,
  "tech": 4
}
fix_label = {
  2: 3,
  4: 2,
  3: 1,
  1: 0,
  0: 4
}
total = defaultdict(lambda: 0)
correct = defaultdict(lambda: 0)

with jsonlines.open("data/gpt2-SetFit__bbc-news.jsonl") as f:
  for item in tqdm(f):
    label = fix_label[item['label']]
    preds = item['prediction']

    total[label] += 5

    for p in preds:
      if label2id[p['label']] == label:
        correct[label] += 1


1225it [00:00, 58334.91it/s]


In [ ]:
for k in sorted(total.keys()):
    print(k, correct[k] / total[k])

3 0.6967272727272728
0 0.8174825174825174
1 0.799047619047619
4 0.6396226415094339
2 0.5884297520661157


In [27]:
correct

defaultdict(<function __main__.<lambda>()>,
            {3: 958, 0: 1169, 1: 839, 4: 678, 2: 712})